In [1]:
# default_exp basic_model

In [2]:
#export
import torch
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from kirby.run_params import RunParams
from kirby.data_manager import DataManager

# Basic Model

> A basic Pytorch lighting module

In [7]:
#export
class BasicModel(pl.LightningModule):
    def __init__(self, run_params):
        super().__init__()
        self.run_params = run_params
        pass

    def prepare_data(self):
        data_manager = DataManager(self.run_params)
        self.train_ds, self.val_ds = data_manager.prepare_data()


    def forward(self, x):
        import pdb; pdb.set_trace()
        loss = self.model(x['input_ids'], attention_mask=x['attention_mask'], labels=x['input_ids'])[0]
        return loss

    def training_step(self, batch, batch_idx):
        loss = self.forward(batch)
        return {'loss': loss, 'log': {'train_loss': loss}}

    def validation_step(self, batch, batch_idx):
        loss = self.forward(batch)
        return {'loss': loss, 'log': {'val_loss': loss}}

    def validation_epoch_end(self, outputs):
        loss = torch.cat([o['loss'].unsqueeze(0) for o in outputs], 0).mean()
        out = {'val_loss': loss}
        return {**out, 'log': out}

    def train_dataloader(self):
        return torch.utils.data.DataLoader(
                self.train_ds,
                batch_size=self.run_params.batch_size,
                drop_last=True,
                shuffle=True,
                num_workers=self.run_params.num_workers,
                pin_memory=True
                ) 

    def val_dataloader(self):
        return torch.utils.data.DataLoader(
                self.val_ds,
                batch_size=self.run_params.batch_size,
                drop_last=True,
                shuffle=False,
                num_workers=self.run_params.num_workers,
                pin_memory=True
                )

    def configure_optimizers(self):
        return torch.optim.SGD(
            self.parameters(),
            lr=self.run_params.lr,
            momentum=self.run_params.momentum,
        ) 

# Testing

In [4]:
# Creation
basic_model = BasicModel(RunParams())
assert isinstance(basic_model, BasicModel)

In [5]:
# Prepare data
basic_model.prepare_data()
from datasets import Dataset
assert isinstance(basic_model.train_ds, Dataset)
assert isinstance(basic_model.val_ds, Dataset)

Using custom data configuration default
Reusing dataset text (/Users/robertmorain/.cache/huggingface/datasets/text/default-4726d5cdfd3031fb/0.0.0/daf90a707a433ac193b369c8cc1772139bb6cca21a9c7fe83bdd16aad9b9b6ab)
Loading cached processed dataset at /Users/robertmorain/.cache/huggingface/datasets/text/default-4726d5cdfd3031fb/0.0.0/daf90a707a433ac193b369c8cc1772139bb6cca21a9c7fe83bdd16aad9b9b6ab/cache-055375cff4ad6c0e.arrow
Using custom data configuration default
Reusing dataset text (/Users/robertmorain/.cache/huggingface/datasets/text/default-4726d5cdfd3031fb/0.0.0/daf90a707a433ac193b369c8cc1772139bb6cca21a9c7fe83bdd16aad9b9b6ab)
Loading cached processed dataset at /Users/robertmorain/.cache/huggingface/datasets/text/default-4726d5cdfd3031fb/0.0.0/daf90a707a433ac193b369c8cc1772139bb6cca21a9c7fe83bdd16aad9b9b6ab/cache-86e72c9a0fe53360.arrow


In [6]:
run_params = RunParams()
trainer = pl.Trainer(
    default_root_dir='logs',
    gpus=(1 if torch.cuda.is_available() else 0),
    max_epochs=run_params.max_epochs,
    fast_dev_run=run_params.debug,
    logger=TensorBoardLogger(save_dir='logs/', name=run_params.run_name),
)

trainer.fit(basic_model)

GPU available: False, used: False
TPU available: None, using: 0 TPU cores
Running in fast_dev_run mode: will run a full train, val and test loop using 1 batch(es).
Using custom data configuration default
Reusing dataset text (/Users/robertmorain/.cache/huggingface/datasets/text/default-4726d5cdfd3031fb/0.0.0/daf90a707a433ac193b369c8cc1772139bb6cca21a9c7fe83bdd16aad9b9b6ab)
Loading cached processed dataset at /Users/robertmorain/.cache/huggingface/datasets/text/default-4726d5cdfd3031fb/0.0.0/daf90a707a433ac193b369c8cc1772139bb6cca21a9c7fe83bdd16aad9b9b6ab/cache-055375cff4ad6c0e.arrow
Using custom data configuration default
Reusing dataset text (/Users/robertmorain/.cache/huggingface/datasets/text/default-4726d5cdfd3031fb/0.0.0/daf90a707a433ac193b369c8cc1772139bb6cca21a9c7fe83bdd16aad9b9b6ab)
Loading cached processed dataset at /Users/robertmorain/.cache/huggingface/datasets/text/default-4726d5cdfd3031fb/0.0.0/daf90a707a433ac193b369c8cc1772139bb6cca21a9c7fe83bdd16aad9b9b6ab/cache-86e72c9

TypeError: 'NoneType' object is not iterable

In [10]:
import multiprocessing
print ('count =', multiprocessing.cpu_count())

count = 8
